In [1]:
import rasterio
from glob import glob
import numpy as np
import pandas as pd
import pickle
import os


In [2]:
##### START OPTIONS #####
root_path = "/data/"
stats_folder = root_path + "band_stats/"
chip_dir_filt = root_path + 'chips_filtered/'
chip_dir_binary_filt = root_path + 'chips_binary_filtered/'
chip_dir_multi_filt = root_path + 'chips_multi_filtered/'

#####  END OPTIONS  #####

In [3]:
dirs_to_make = [stats_folder]
for folder in dirs_to_make:
    try:
        os.makedirs(folder)
    except FileExistsError:
        # directory already exists
        print('pass')
        pass

pass


read in values for all HLS bands, one band at a time. save values on a per band basis

In [6]:
chips = glob(chip_dir_filt + "*_merged.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 19):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
    #    print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'band_values_' + str(band) + '.list', 'wb') as file:
        pickle.dump(vals_all, file)
        



        
    #out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)


4239
1
447.6676168488007
171.40573643186065
2
706.7407293818094
230.25873139935956
3
879.0833726852604
350.9951097428933
4
2184.2076620844286
635.5527779445017
5
2606.9293695580564
795.3758705776565
6
1770.841471899546
643.8712791473844
7
349.5615936362602
159.36729877065247
8
641.0655599061768
196.9676638857457
9
615.8356092757786
321.9486206870691
10
2742.278495651899
653.353139790775
11
2196.7776814478334
681.781172524255
12
1272.224865439917
586.9196411782743
13
336.7200042169093
161.5689569541235
14
612.4113323178374
207.076470789683
15
611.6312338530157
352.79593826985223
16
2795.0618147640007
595.1689886161289
17
2229.998832940322
758.8212282455588
18
1253.424746200287
662.8159664512201


In [7]:
print(means)
print(stds)

[447.6676168488007, 706.7407293818094, 879.0833726852604, 2184.2076620844286, 2606.9293695580564, 1770.841471899546, 349.5615936362602, 641.0655599061768, 615.8356092757786, 2742.278495651899, 2196.7776814478334, 1272.224865439917, 336.7200042169093, 612.4113323178374, 611.6312338530157, 2795.0618147640007, 2229.998832940322, 1253.424746200287]
[171.40573643186065, 230.25873139935956, 350.9951097428933, 635.5527779445017, 795.3758705776565, 643.8712791473844, 159.36729877065247, 196.9676638857457, 321.9486206870691, 653.353139790775, 681.781172524255, 586.9196411782743, 161.5689569541235, 207.076470789683, 352.79593826985223, 595.1689886161289, 758.8212282455588, 662.8159664512201]


save band means and sds to csv

In [8]:
band_names = list(range(1, 19))
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds.csv")

consolidate means, sds for dates

In [9]:
means = []
stds = []

for item in [[1,7,13], [2,8,14], [3,9,15], [4,10,16], [5, 11, 17], [6, 12, 18]]:
    print(item)
    band_vals = []
    for band in item:
        print(band)
        with open(stats_folder + 'band_values_' + str(band) + '.list', 'rb') as f:
            x = pickle.load(f)
            print(x.mean())
            print(x.std())
            band_vals.append(x)
            del x

    band_vals = np.concatenate((band_vals[0], band_vals[1], band_vals[2]))
    mean = band_vals.mean()
    std = band_vals.std()
    means.append(mean)
    stds.append(std)
    del band_vals
        
            

[1, 7, 13]
1
447.6676168488007
171.40573643186065
7
349.5615936362602
159.36729877065247
13
336.7200042169093
161.5689569541235
[2, 8, 14]
2
706.7407293818094
230.25873139935956
8
641.0655599061768
196.9676638857457
14
612.4113323178374
207.076470789683
[3, 9, 15]
3
879.0833726852604
350.9951097428933
9
615.8356092757786
321.9486206870691
15
611.6312338530157
352.79593826985223
[4, 10, 16]
4
2184.2076620844286
635.5527779445017
10
2742.278495651899
653.353139790775
16
2795.0618147640007
595.1689886161289
[5, 11, 17]
5
2606.9293695580564
795.3758705776565
11
2196.7776814478334
681.781172524255
17
2229.998832940322
758.8212282455588
[6, 12, 18]
6
1770.841471899546
643.8712791473844
12
1272.224865439917
586.9196411782743
18
1253.424746200287
662.8159664512201


In [10]:
print(means)
print(stds)

[377.9830715673234, 653.4058738686078, 702.1834052713516, 2573.849324166776, 2344.5686279820707, 1432.16369451325]
[171.51165872566156, 215.5407551216428, 364.3545395986149, 686.5730746423333, 769.6448444305327, 675.9192684236534]


In [11]:
band_names = ['B02', 'B03', 'B04', 'B8A', 'B11', 'B12']
summary_df = pd.DataFrame({"band": band_names  ,  "mean": means, "sd": stds})
summary_df
summary_df.to_csv(stats_folder + "means_sds_consolidated_dates.csv")

calculate counts for all CDL classes

In [12]:
chips = glob(chip_dir_filt +  "*.mask.tif")
print(len(chips))
means = []
stds = []

for band in range(1, 2):
    print(band)
    vals_all = np.ndarray([0])
    for k in range(len(chips)):
     #   print(k)
        file = chips[k]
        with rasterio.open(file) as src:
            vals = src.read(band).flatten()
            if(len(vals) != 50176):
                print(k)
                print(file)
                print(len(vals))
            vals_all = np.concatenate([vals_all, vals])
    mean = vals_all.mean()
    std = vals_all.std()
    print(mean)
    print(std)
    means.append(mean)
    stds.append(std)
    
    with open(stats_folder + 'CDL_values.list', 'wb') as file:
        pickle.dump(vals_all, file)
        


4239
1
140.65953471052478
48.327096840758394


export CDL counts 

In [15]:
len(vals_all)
unique, counts = np.unique(vals_all, return_counts=True)
total_count = counts.sum()
pcts = (100.0*counts)/total_count
freq_table =  pd.DataFrame({"CDL_val": unique  ,  "count": counts, "pct": pcts})

In [16]:
freq_table.to_csv(stats_folder + "cdl_freq.csv")

In [17]:
counts.sum()

212696064